In [7]:
#transformation du texte sous forme de liste en string
def jonction(liste):
  result = ""
  for token in liste:
    result = result + token + " "
  return result

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from joblib import load,dump
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
import time
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def optimisation_model_avec_regex(df_initial,colonne_a_vectoriser,liste_colonne_regex,max_features,k_best,joblib_path_suivi_metrique,grid_search_cv):
    #import du df
    df = load(df_initial)
    #transformation de la future colonne feature en chaine de caractère
    df[colonne_a_vectoriser]=df[colonne_a_vectoriser].apply(jonction)
    #séparation des features et de la cible
    liste_colonne_a_vectoriser = [colonne_a_vectoriser]
    data = df[liste_colonne_a_vectoriser+liste_colonne_regex]
    target = df["prdtypecode"]
    #séparation de train et test
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42, stratify = target)
    #vectorisation selon un nombre max de mots les plus fréquents
    vectorizer = CountVectorizer(max_features=max_features)
    X_train_regex = X_train[liste_colonne_regex]
    X_train_vec = pd.DataFrame(vectorizer.fit_transform(X_train[colonne_a_vectoriser]).todense(),index=X_train_regex.index)
    X_train = pd.concat([X_train_regex,X_train_vec],axis = 1)
    X_test_regex = X_test[liste_colonne_regex]
    X_test_vec = pd.DataFrame(vectorizer.transform(X_test[colonne_a_vectoriser]).todense(),index=X_test_regex.index)
    X_test = pd.concat([X_test_regex,X_test_vec],axis = 1)
    X_train.columns = X_train.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    scaler = StandardScaler().fit(np.asarray(X_train))
    X_train = scaler.transform(np.asarray(X_train))
    X_test = scaler.transform(np.asarray(X_test))
    sel = SelectKBest(k=k_best)
    sel.fit(np.asarray(X_train),y_train)
    mask = sel.get_support()
    X_train = sel.transform(np.asarray(X_train))
    X_test = sel.transform(np.asarray(X_test))
    if os.path.exists(joblib_path_suivi_metrique):
        df_import = load(joblib_path_suivi_metrique)
        print("récupération du df existant")
    else:
        df_import = pd.DataFrame(columns=["Max_features","K_best","Grid","Hyperparamètres","Best_param", "Accuracy", "F1_weighted", "F1_macro", "Duree en sec"])
        print("création d'un dataframe")
    score = []
    print("debut du grid:",grid_search_cv)
    debut = time.time()
    grid_search_cv.fit(np.asarray(X_train),y_train)
    print("fin du grid:",grid_search_cv)
    print("debut meilleur param:",grid_search_cv.best_params_)
    best_params = grid_search_cv.best_params_
    best_rf_model = RandomForestClassifier(**best_params)
    best_rf_model.fit(np.asarray(X_train), y_train)
    y_pred  = best_rf_model.predict(np.asarray(X_test))
    print("fin meilleur param")
    accuracy = accuracy_score(y_test, y_pred)
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    fin = time.time()
    duree = fin - debut
    model_scores = { 
            "Max_features": f"{max_features}",
            "K_best":f"{k_best}",
            "Grid": f"{grid_search_cv}",
            "Hyperparamètres":f"{param_grid}",
            "Best_param":f"{best_params}",
            "Accuracy": accuracy,
            "F1_weighted": f1_weighted,
            "F1_macro": f1_macro,
            "Duree en sec": duree}
    score.append(model_scores)
    df_score = pd.DataFrame(score)
    df = pd.concat([df_import, df_score], ignore_index=True)
    return df

Verification de l'importance des colonnes regex 

In [43]:
df_initial = r"C:\Users\franc\AutoML\df_post_trad_et_regex.joblib"
colonne_a_vectoriser = "mots_stem_sans_chiffres"
liste_colonne_regex = ['masse_encod_freq', 'surface_encod_freq', 'vitesse_encod_freq',
       'pression_encod_freq', 'energie_alim_encod_freq',
       'energie_elec_encod_freq', 'longueur_encod_freq', 'volume_encod_freq',
       'memoire_encod_freq', 'temps_encod_freq', 'chiffres_encod_freq',
       'description_vide_encod_freq', 'Nb_mots_cat_encod_freq']
max_features = 4000
k_best = 3000
df = load(df_initial)
#transformation de la future colonne feature en chaine de caractère
df[colonne_a_vectoriser]=df[colonne_a_vectoriser].apply(jonction)
#séparation des features et de la cible
liste_colonne_a_vectoriser = [colonne_a_vectoriser]
data = df[liste_colonne_a_vectoriser+liste_colonne_regex]
target = df["prdtypecode"]
#séparation de train et test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42, stratify = target)
#vectorisation selon un nombre max de mots les plus fréquents
vectorizer = CountVectorizer(max_features=max_features)
X_train_regex = X_train[liste_colonne_regex]
X_train_vec = pd.DataFrame(vectorizer.fit_transform(X_train[colonne_a_vectoriser]).todense(),index=X_train_regex.index)
X_train = pd.concat([X_train_regex,X_train_vec],axis = 1)
X_test_regex = X_test[liste_colonne_regex]
X_test_vec = pd.DataFrame(vectorizer.transform(X_test[colonne_a_vectoriser]).todense(),index=X_test_regex.index)
X_test = pd.concat([X_test_regex,X_test_vec],axis = 1)
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns)
sel = SelectKBest(k=k_best)
sel.fit(X_train,y_train)
mask = sel.get_support()
X_train_sel = sel.transform(X_train)
X_test_sel = sel.transform(X_test)  
    
liste_columns_retenues = []
for i,j in enumerate(mask):
    if j:
        liste_columns_retenues.append(X_train.columns[i])  


s=0
for i in liste_colonne_regex:
    if i in liste_columns_retenues:
        s+=1
print(f' Sur les 13 colonnes de regex on en retient :' ,{s})

 Sur les 13 colonnes de regex on en retient : {12}


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rdf = RandomForestClassifier(n_jobs = -1,random_state = 23)

param_grid = {
    'n_estimators': [50],  # Nombre d'arbres dans la forêt
    'criterion': ["log_loss"],  # Métrique de qualité de la division
    'max_depth': [None],  # Profondeur maximale de chaque arbre
    'min_samples_split': [5],  # Nombre minimal d'échantillons requis pour diviser un nœud
    'min_samples_leaf': [1],  # Nombre minimal d'échantillons requis pour être une feuille
    'max_features': ['log2'],  # Nombre maximal de fonctionnalités à considérer pour la division
    'bootstrap': [False],  # Si l'échantillonnage bootstrap doit être utilisé
    'class_weight': ['balanced'],  # Poids des classes pour la correction de déséquilibre
    'random_state': [23],  # Seed pour la reproductibilité
    'n_jobs': [-1]  # Utilisation de tous les cœurs de CPU}
}


gcv = GridSearchCV(estimator=rdf, param_grid = param_grid,cv=5, n_jobs=-1, scoring="f1_weighted")

In [11]:
df_initial = r"C:\Users\franc\AutoML\df_post_trad_et_regex.joblib"
colonne_a_vectoriser = "mots_stem_sans_chiffres"
liste_colonne_regex = ['masse_encod_freq', 'surface_encod_freq', 'vitesse_encod_freq',
       'pression_encod_freq', 'energie_alim_encod_freq',
       'energie_elec_encod_freq', 'longueur_encod_freq', 'volume_encod_freq',
       'memoire_encod_freq', 'temps_encod_freq', 'chiffres_encod_freq',
       'description_vide_encod_freq', 'Nb_mots_cat_encod_freq']
max_features = 4000
k_best = 3000
joblib_path_suivi_metrique = r"C:\Users\franc\AutoML\df_optimisation_avec_regex.joblib"
grid_search_cv = gcv
df_optimisation_avec_regex = optimisation_model_avec_regex(df_initial,colonne_a_vectoriser,liste_colonne_regex,max_features,k_best,joblib_path_suivi_metrique,grid_search_cv)
dump(df_optimisation_avec_regex,r"C:\Users\franc\AutoML\df_optimisation_avec_regex.joblib")

création d'un dataframe
debut du grid: GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1, random_state=23),
             n_jobs=-1,
             param_grid={'bootstrap': [False], 'class_weight': ['balanced'],
                         'criterion': ['log_loss'], 'max_depth': [None],
                         'max_features': ['log2'], 'min_samples_leaf': [1],
                         'min_samples_split': [5], 'n_estimators': [50],
                         'n_jobs': [-1], 'random_state': [23]},
             scoring='f1_weighted')


C:\Users\franc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
1 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\franc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\franc\anaconda3\lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\franc\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 348, in fit
    X, y = self._validate_data(
  File "C:\Users\franc\anaconda3\lib\site-

fin du grid: GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1, random_state=23),
             n_jobs=-1,
             param_grid={'bootstrap': [False], 'class_weight': ['balanced'],
                         'criterion': ['log_loss'], 'max_depth': [None],
                         'max_features': ['log2'], 'min_samples_leaf': [1],
                         'min_samples_split': [5], 'n_estimators': [50],
                         'n_jobs': [-1], 'random_state': [23]},
             scoring='f1_weighted')
debut meilleur param: {'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50, 'n_jobs': -1, 'random_state': 23}
fin meilleur param


['C:\\Users\\franc\\AutoML\\df_optimisation_avec_regex.joblib']

In [12]:
df_optimisation_avec_regex

,Max_features,K_best,Grid,Hyperparamètres,Best_param,Accuracy,F1_weighted,F1_macro,Duree en sec
0,4000,3000,"GridSearchCV(cv=5, estimator=RandomForestClass...","{'n_estimators': [50], 'criterion': ['log_loss...","{'bootstrap': False, 'class_weight': 'balanced...",0.775329,0.773499,0.755715,174.347588
